In [18]:
# Prep ... import libraries
import pandas as pd
import requests
from pathlib import Path

# Import API key & set base URL
from api_keys import walkscore_key
url="https://api.walkscore.com/score?format=json&address=skip"

# Define variables
FindState = "WA"

In [19]:
# Import CSV
data_df = pd.read_csv("../resources/obesity_data_ALL_states.csv")
data_df = pd.DataFrame(data_df)

# Insert new column at end to import Walk Score
data_df["Walk Score"]=''

# Clean up dataframe by removing any rows with no data
data_df_cleaned=data_df.dropna()

# data_df_cleaned["StateAbbr"].value_counts()

In [21]:
# Let's do API calls for just ONE state
# We define FindState in first block...
one_state = data_df_cleaned.loc[data_df_cleaned["StateAbbr"] == FindState, :]

# reset index because of dropped rows
one_state.reset_index(drop=True, inplace=True)

one_state

,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,WA,Seattle,5363000-53033006300,5363000,53033006300,19.5,4962,47.636205,-122.275885,
1,WA,Tacoma,5370000-53053062801,5370000,53053062801,32.9,6130,47.224245,-122.497173,
2,WA,Bellevue,5305210-53033024902,5305210,53033024902,19.8,2601,47.564329,-122.134710,
3,WA,Kent,5335415-53033029206,5335415,53033029206,30.3,3853,47.413149,-122.243568,
4,WA,Vancouver,5374060-53011041310,5374060,53011041310,23.5,4746,45.596718,-122.518321,
...,...,...,...,...,...,...,...,...,...,...
498,WA,Seattle,5363000-53033001600,5363000,53033001600,21.5,4131,47.700659,-122.382500,
499,WA,Tacoma,5370000-53053062600,5370000,53053062600,32.3,2977,47.219741,-122.476116,
500,WA,Kennewick,5335275-53005011504,5335275,53005011504,30.1,2312,46.167351,-119.125996,
501,WA,Spokane,5367000-53063000500,5367000,53063000500,32.8,3403,47.707871,-117.418497,


In [22]:
# Now let's iterate thru that ONE state and get the data

for index,row in one_state.iterrows():

    try:
        lat = str(row["Lat"])
        lon = str(row["Lon"])
        query_url = url+"&lat="+lat+"&lon="+lon+"&wsapikey="+walkscore_key
        walk_response = requests.get(query_url)
        walk_json = walk_response.json() 
        if walk_json["status"] == 41:
            print("Daily API quota exceeded")
            print("^^^^^^^^^^^^^^^^^^^^^^^^")
            break
            
        print("Processing LAT %s, LON, %s" % (lat, lon))
        one_state.loc[index, "Walk Score"] = walk_json["walkscore"]
    
    except:
        print("Data not found. Skipping")
        pass


Processing LAT 47.636204596, LON, -122.275885393
Processing LAT 47.2242452813, LON, -122.497172711
Processing LAT 47.5643288754, LON, -122.134709858
Processing LAT 47.4131488108, LON, -122.24356832
Processing LAT 45.5967183312, LON, -122.518320907
Processing LAT 47.9417269181, LON, -122.203880637
Processing LAT 47.4503896809, LON, -122.178457056
Processing LAT 47.7067275502, LON, -122.366496857
Processing LAT 47.1939936737, LON, -122.496412414
Processing LAT 47.2791184551, LON, -122.155070317
Processing LAT 47.5537877711, LON, -122.310225765
Processing LAT 47.6717888713, LON, -117.400191522
Processing LAT 47.6658472901, LON, -117.271911536
Processing LAT 47.6051125357, LON, -122.324946395
Processing LAT 47.6589661327, LON, -122.3230895
Processing LAT 47.6645733026, LON, -117.208769317
Processing LAT 47.7078416216, LON, -117.387014933
Processing LAT 47.692321988, LON, -117.371213698
Processing LAT 47.3026458556, LON, -122.404407882
Processing LAT 46.6134286531, LON, -120.598333706
Proce

In [16]:
one_state.head()


,StateAbbr,CityName,UniqueID,CityFIPS,TractFIPS,Data_Value,Population2010,Lat,Lon,Walk Score
0,OH,Parma,3961000-39035177404,3961000,39035177404,29.2,2875,41.389745,-81.739448,56
1,OH,Columbus,3918000-39049000810,3918000,39049000810,37.8,2540,40.026543,-82.976199,47
2,OH,Columbus,3918000-39049008825,3918000,39049008825,35.1,5908,39.886204,-82.977271,25
3,OH,Toledo,3977000-39095008400,3977000,39095008400,32.2,758,41.667167,-83.683257,53
4,OH,Dayton,3921000-39113165200,3921000,39113165200,48.8,1669,39.767184,-84.198173,65


In [17]:
# Save output to CSV
output_csv = f'../resources/{FindState}_complete.csv'
one_state.to_csv(output_csv, index=False)